In [2]:
import pandas as pd

In [3]:
risk = pd.read_csv("/Users/berelsom/Library/CloudStorage/OneDrive-NorwichBioScienceInstitutes/Pathogen_Database/Pathogen_Database_080524/Risk_Register_05_24.csv")
# risk.columns

Looking at the information in each column

In [20]:
def unique_entries(df, column=None):
    if column:
        if column in df.columns:
            return {column: df[column].unique()}
        else:
            raise ValueError(f"Column '{column}' does not exist in the DataFrame.")
    else:
        unique_dict = {}
        for col in df.columns:
            unique_dict[col] = df[col].unique()
        return unique_dict

# To print unique values
unique_values = unique_entries(risk, column='General Comments')
# unique_values

Process to remove columns which are also removed during database creation to keep the output smaller

In [4]:
# Fill NaN values with empty strings
risk['Type of pest'] = risk['Type of pest'].fillna("")  
# Define the list of items to remove
remove = ["Insect", "Mite", "Nematode", "Plant"] 
# Filter rows where 'Type of pest' is not in the remove list
risk = risk[~risk['Type of pest'].isin(remove)] 
# Remove single quotes from 'Pest Name' column if present
risk['Pest Name'] = risk['Pest Name'].str.replace("'", "")

# Selecting columns
columns_keep = [
    'Type of pest', 'Pest Name', 'EU and EPPO listing', 'UK', 'Pathways', 'Likelihood', 'Impact ',
    'UK Relative Risk Rating (unmitigated)', 'Regulation', 'Likelihood.1', 'Impact .1',
     'UK Relative Risk Rating (mitigated)', 'Scenario for Risk Register'
    ]
risk = risk[columns_keep]

# Create a new column 'Species' with only the Genus and Species
risk['Species'] = risk['Pest Name'].apply(lambda x: ' '.join(x.split()[:2]))

# Create a new column 'Regulated' based on the condition
risk['Regulated'] = risk['EU and EPPO listing'].apply(
    lambda x: 'Yes' if isinstance(x, str) and 'regulated quarantine pest' in x.lower() else 'No'
)

# Create a new column 'Natural Spread' based on the condition
risk['Natural Spread'] = risk['Pathways'].apply(
    lambda x: 'Yes' if isinstance(x, str) and 'natural spread' in x.lower() else 'No'
)

 # Rename some columns 
risk = risk.rename(columns={
    'Likelihood': 'Likelihood (unmitigated)',
    'Likelihood.1': 'Likelihood (mitigated)',
    'Impact ': 'Impact (unmitigated)',
    'Impact .1': 'Impact (mitigated)',
    'UK Relative Risk Rating (unmitigated)': 'Risk Rating (unmitigated)',
    'UK Relative Risk Rating (mitigated)': 'Risk Rating (mitigated)'
})

# Drop the old columns
risk = risk.drop(columns=['EU and EPPO listing', 'Pathways'])

In [5]:
risk

,Type of pest,Pest Name,UK,Likelihood (unmitigated),Impact (unmitigated),Risk Rating (unmitigated),Regulation,Likelihood (mitigated),Impact (mitigated),Risk Rating (mitigated),Scenario for Risk Register,Species,Regulated,Natural Spread
0,Bacterium,Candidatus Liberibacter africanus,Absent,NaN,NaN,1.0,Regulated quarantine pest (NI only),NaN,NaN,1.0,Pest is introduced,Candidatus Liberibacter,Yes,No
1,Bacterium,Candidatus Liberibacter americanus,Absent,NaN,NaN,1.0,QP (NI only). Prohibition of key pathway.,NaN,NaN,1.0,Pest is introduced,Candidatus Liberibacter,Yes,No
2,Bacterium,Candidatus Liberibacter asiaticus,Absent,0.0,0.0,1.0,Regulated quarantine pest (NI),0.0,0.0,1.0,Pest is introduced,Candidatus Liberibacter,Yes,No
3,Bacterium,Candidatus Phlomobacter fragariae,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Candidatus Phlomobacter,No,No
4,Phytoplasma,Candidatus Phytoplasma americanum,Absent,NaN,NaN,NaN,Regulated quarantine pest (Northern Ireland),NaN,NaN,NaN,NaN,Candidatus Phytoplasma,Yes,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1383,Bacterium,Xanthomonas vesicatoria,Absent,5.0,4.0,80.0,Regulated non quarantine pest,2.0,3.0,24.0,Pest is introduced,Xanthomonas vesicatoria,No,No
1404,Bacterium,Xylella fastidiosa,Absent,4.0,3.0,60.0,QP for both GB and NI. Import regulations on p...,2.0,3.0,30.0,Pest is introduced,Xylella fastidiosa,Yes,No
1405,Bacterium,Xylella taiwanensis,Absent,3.0,3.0,45.0,NaN,2.0,3.0,30.0,Pest is introduced,Xylella taiwanensis,No,No
1406,Bacterium,Xylophilus ampelinus,Absent,2.0,4.0,16.0,QP GB; RNQP NI,2.0,4.0,16.0,Pest is introduced,Xylophilus ampelinus,No,No
